# Concat multiple iterations of sampling for coherent layout in Gephi

In [39]:
import numpy as np
import pandas as pd
import networkx as nx

from fairpair import *

In [40]:
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [41]:
H = nx.MultiDiGraph()
H.add_nodes_from(G.nodes(data=True))

In [42]:
for iteration in range(10):
    sampling = ProbKnockoutSampling(G, warn=False)
    sampling.apply(iter=1, k=1)
    # get the current graph and add it to the multigraph, but label edges by iteration
    edges = [(i, j, {**data, 'iteration': iteration}) for i, j, data in G.edges(data=True)]
    H.add_edges_from(edges)
    print(len(H.edges))

8
20
36
52
72
92
112
134
158
182


In [43]:
H.edges(data=True)

OutMultiEdgeDataView([(0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 0}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 1}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 2}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 3}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 4}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 5}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 6}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 7}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 8}), (0, 7, {'wins': 1, 'weight': 1.0, 'iteration': 9}), (1, 11, {'wins': 0, 'weight': 0.0, 'iteration': 7}), (1, 11, {'wins': 0, 'weight': 0.0, 'iteration': 8}), (1, 11, {'wins': 0, 'weight': 0.0, 'iteration': 9}), (1, 9, {'wins': 0, 'weight': 0.0, 'iteration': 8}), (1, 9, {'wins': 0, 'weight': 0.0, 'iteration': 9}), (4, 9, {'wins': 1, 'weight': 1.0, 'iteration': 0}), (4, 9, {'wins': 1, 'weight': 1.0, 'iteration': 1}), (4, 9, {'wins': 1, 'weight': 1.0, 'iteration': 2}), (4, 9, {'wins': 1, 'weight': 1.0, 'iter

In [44]:
for node in H.nodes(data=True):
    node[1]['minority'] = bool(node[1]['minority']) # convert from numpy
nx.write_gexf(H, 'data/multigraphtest.gexf')